In [1]:
import torchvision
import torch.nn
import torch.utils.tensorboard

import torchvision.transforms
from torch.utils.data import DataLoader

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# 准备数据集
train_data = torchvision.datasets.CIFAR10("../CIFAR10", train=True,
                                          transform=torchvision.transforms.ToTensor(), download=True)
# cifar10是默认piltenser，要转化为pytorchtensor

test_data = torchvision.datasets.CIFAR10("../CIFAR10", train=False, download=True,
                                         transform=torchvision.transforms.ToTensor())
# print(len(train_data))
# print(len(test_data))
train_data_size = len(train_data)
test_data_size = len(test_data)
# 用dataloader加载数据集,加载为一组64个
train_dataloader = DataLoader(dataset=train_data, batch_size=64)
test_dataloader = DataLoader(dataset=test_data, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# 添加tensorboard可视化
writer = torch.utils.tensorboard.SummaryWriter("./logs")

In [4]:
# 搭建神经网络
class CIFAR10Classify01(torch.nn.Module):
    def __init__(self):
        super(CIFAR10Classify01, self).__init__()
        self.model1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, padding=2, ),
            torch.nn.MaxPool2d(kernel_size=2),
            torch.nn.BatchNorm2d(32),
            torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=2, ),
            torch.nn.MaxPool2d(kernel_size=2),
            torch.nn.Sigmoid(),
            torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=2, ),
            torch.nn.MaxPool2d(2),
            torch.nn.Flatten(),
            # 第一个linear的in_features可以由print(x.shape)得到
            torch.nn.Linear(in_features=1024, out_features=64),
            torch.nn.Linear(in_features=64, out_features=10)
            # 分类问题的最终out_features和类数相同
        )

    def forward(self, x):
        x = self.model1(x)
        return x
    
myModel = CIFAR10Classify01()
myModel.to(device)

CIFAR10Classify01(
  (model1): Sequential(
    (0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Sigmoid()
    (6): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Flatten(start_dim=1, end_dim=-1)
    (9): Linear(in_features=1024, out_features=64, bias=True)
    (10): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [5]:
# 定义损失函数
loss_fn = torch.nn.CrossEntropyLoss()
loss_fn.to(device)

# 定义优化器
learning_rate = 1e-2
optim = torch.optim.SGD(params=myModel.parameters(), lr=learning_rate)
# SGD随机梯度下降

In [6]:
# 设置训练网络参数
train_step = 0
test_step = 0
epoch = 100
for i in range(epoch):
    train_loss = 0
    print("------{}------".format(i))
    myModel.train()
    for imgs, targets in train_dataloader:
        imgs = imgs.to(device)
        targets = targets.to(device)
        output = myModel(imgs)
        # 优化器梯度清零
        optim.zero_grad()
        # 计算损失
        loss = loss_fn(output, targets)
        # 反向传播算梯度
        loss.backward()
        # 优化器优化模型
        optim.step()
        train_loss += loss.item()
        train_step += 1
        # print(loss)
    writer.add_scalar("trainloss",  train_loss/len(train_data), train_step)
    print("running_loss: ", train_loss/len(train_data))

    # 验证每轮学习效果
    test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        # 取消梯度，验证时不调优
        myModel.eval()
        for imgs, targets in test_dataloader:
            imgs = imgs.to(device)
            targets = targets.to(device)
            output = myModel(imgs)
            loss = loss_fn(output, targets)
            test_loss += loss.item()
            accuracy = (output.argmax(1) == targets).sum()
            total_accuracy += accuracy
            test_step += 1
    writer.add_scalar("testloss", test_loss/len(test_data), test_step)
    writer.add_scalar("total_accuracy", total_accuracy/len(test_data), test_step)
    print("test_loss: ", test_loss/len(test_data))
    print("total_accuracy: ", total_accuracy/len(test_data))
    torch.save(myModel, "./myModel_{}.pth".format(i))

writer.close()

------0------


RuntimeError: running_mean should contain 3 elements not 32

In [ ]:
%reload_ext tensorboard


In [ ]:

from tensorboard import notebook
notebook.list()
notebook.start("--logdir ./logs --port=709")